In [1]:
import datetime


from cogment_lab.actors import RandomActor, ConstantActor
from cogment_lab.envs.pettingzoo import AECEnvironment
from cogment_lab.process_manager import Cogment
from cogment_lab.utils.runners import process_cleanup


In [2]:
# Cleans up potentially hanging background processes from previous runs
process_cleanup()

Processes terminated successfully.


In [3]:
logpath = f"logs/logs-{datetime.datetime.now().isoformat()}"

cog = Cogment(log_dir=logpath)

print(logpath)

logs/logs-2023-12-13T22:21:48.548102


/Users/ariel/PycharmProjects/cogment_lab/venv/lib/python3.10/site-packages/cogment/context.py:213: UserWarning: No logging handler defined (e.g. logging.basicConfig)
  warnings.warn("No logging handler defined (e.g. logging.basicConfig)")


In [4]:
# Launch an environment in a subprocess

cenv = AECEnvironment(env_path="pettingzoo.butterfly.knights_archers_zombies_v10.env",
                      render=True)

await cog.run_env(env=cenv, 
                  env_name="kaz",
                  port=9001, 
                  log_file="env.log")


True

In [5]:
# Launch two dummy actors in subprocesses

random_actor = RandomActor(cenv.env.action_space("knight_0"))
constant_actor = ConstantActor(5)

await cog.run_actor(actor=random_actor, 
                    actor_name="random", 
                    port=9021, 
                    log_file="actor-random.log")

await cog.run_actor(actor=constant_actor,
                    actor_name="constant",
                    port=9022,
                    log_file="actor-constant.log")


True

In [6]:
# Check what's running
cog.processes

{'kaz': <SpawnProcess name='SpawnProcess-1' pid=19578 parent=19541 started>,
 'random': <SpawnProcess name='SpawnProcess-2' pid=19579 parent=19541 started>,
 'constant': <SpawnProcess name='SpawnProcess-3' pid=19582 parent=19541 started>}

In [7]:
PONG_ACTIONS = ["ArrowUp", "ArrowDown", "ArrowLeft", "ArrowRight", "f", "no-op"]

actions = PONG_ACTIONS
await cog.run_web_ui(actions=actions, log_file="human.log", fps=60*4)

True

In [9]:
# Get data from a random + random trial
# You can change the values in `actor_impls` between `web_ui`, `random`, and `constant` to see the different behaviors

trial_id = await cog.start_trial(
    env_name="kaz",
    session_config={"render": True},
    actor_impls={
        "knight_0": "random",
        "knight_1": "random",
        "archer_0": "web_ui",
        "archer_1": "random",
    },
)

In [10]:

data = await cog.get_trial_data(trial_id)


In [15]:
data["knight_0"].actions

array([4, 0, 2, 4, 3, 2, 3, 2, 5, 2, 0, 5, 0, 0, 1, 2, 4, 3, 2, 5, 4, 5,
       3, 1, 5, 0, 2, 4, 5, 0, 4, 1, 2, 5, 5, 0, 1, 5, 0, 2, 5, 4, 3, 4,
       2, 2, 2, 1, 2, 2, 4, 0, 2, 3, 2, 4, 2, 1, 3, 5, 1, 4, 0, 4, 5, 1,
       5, 4, 3, 4, 1, 5, 0, 4, 4, 2, 0, 5, 2, 5, 2, 2, 4, 3, 4, 3, 2, 0,
       3, 0, 4, 3, 3, 3, 1, 0, 1, 0, 0, 1, 3, 3, 2, 1, 1, 3, 2, 3, 1, 5,
       5, 3, 4, 5, 5, 3, 0, 0, 5, 5, 4, 2, 3, 2, 2, 5, 0, 5, 0, 0, 3, 3,
       3, 5, 3, 4, 3, 2, 0, 0, 0, 4, 1, 1, 1, 0, 1, 0, 2, 5, 0, 4, 5, 0,
       3, 0, 4, 1, 1, 2, 2, 0, 5, 0, 0, 0, 1, 5, 4, 0, 0, 5, 2, 0, 1, 5,
       3, 2, 3, 5, 4, 0])